In [ ]:
from elasticsearch import Elasticsearch
import wikipedia
import re

In [ ]:
client = Elasticsearch()
indexName = "medical"
client.indices.create(index = indexName)

In [ ]:
diseaseMapping = {
    'properties': {
        'name': {'type': 'string'},
        'title': {'type': 'string'},
        'fulltext': {'type': 'string'}
    }
}
client.indices.put_mapping(index = indexName, doc_type = 'diseases', body = diseaseMapping)

In [ ]:
dl = wikipedia.page("Lists_of_diseases")
dl.links

In [ ]:
diseaseListArray = []
diseaseListArray.append(wikipedia.page("List_of_diseases_(B)"))
#diseaseListArray.append(wikipedia.page("List_of_diseases_(Z)"))
print(diseaseListArray)
#for link in dl.links[15:16]:
#    print("link: ", link)
#    try:
#        diseaseListArray.append(wikipedia.page(link))
#        print(wikipedia.page(link).title)
#        print(wikipedia.page(link).content)
#    except Exception:
#        print("Exception Occurred")
#diseaseListArray
docType = 'diseases'
for diseaselistNumber, diseaselist in enumerate(diseaseListArray):
    for disease in diseaselist.links:
        print("diseaselistNumber: ", diseaselistNumber)
        try:
            if disease[0:4] != "List":
                currentPage = wikipedia.page(disease)
                print("name: ", disease)
                print("title: ", currentPage.title)
#                print("fullText: ", currentPage.content)
                client.index(index=indexName, doc_type=docType,id = disease, body={"name": disease,
                                 "title": currentPage.title, "fulltext": currentPage.content})
        except Exception as ex:
            print(ex)


In [10]:
indexName = "medical"
docType="diseases"
searchFrom = 0
searchSize= 3

searchBody = {
    "stored_fields": ["name"],
    "query": {
        "simple_query_string": {
            "query": '+fatigue+fever+"joint pain"',
            "fields": ["fulltext","title^5","name^10"]
        }
    }
}
client.search(index = indexName, doc_type = docType, body = searchBody, from_ = searchFrom, size = searchSize)

{'took': 3,
 'timed_out': False,
 '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0},
 'hits': {'total': 18,
  'max_score': 7.3687305,
  'hits': [{'_index': 'medical',
    '_type': 'diseases',
    '_id': 'Arthritis, juvenile',
    '_score': 7.3687305},
   {'_index': 'medical',
    '_type': 'diseases',
    '_id': 'Arthritis',
    '_score': 7.297332},
   {'_index': 'medical',
    '_type': 'diseases',
    '_id': 'Acute gouty arthritis',
    '_score': 5.8312016}]}}